In [61]:
# All the libraries needed 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

________________________________________________________

# Q1 How to perform one hot encoding for multi categorical variables

In [76]:
  df = pd.read_csv("Data/benz/train.csv",usecols=["X1","X2","X3","X4","X5","X6"])

In [63]:
df.head()

,X1,X2,X3,X4,X5,X6
0,v,at,a,d,u,j
1,t,av,e,d,y,l
2,w,n,c,d,x,j
3,t,n,f,d,x,l
4,v,n,f,d,h,d


In [64]:
# How many categories are in each column
for col in df.columns:
    print(col,": ",len(df[col].unique())," labels")

X1 :  27  labels
X2 :  44  labels
X3 :  7  labels
X4 :  4  labels
X5 :  29  labels
X6 :  12  labels


In [65]:
# Lets see how many columns we get after OHC
pd.get_dummies(df,drop_first=True).shape

# There are now total 117 variables
# What can we do instead?

(4209, 117)

In [66]:
# We can take the most frequent categories from the data to get the 
# lets take only 20 categories
df.X1.value_counts().sort_values(ascending=False).head(20)

aa    833
s     598
b     592
l     590
v     408
r     251
i     203
a     143
c     121
o      82
w      52
z      46
u      37
e      33
m      32
t      31
h      29
f      23
y      23
j      22
Name: X1, dtype: int64

In [67]:
# lets get the to 10 categories in X2
top_10 = df.X2.value_counts().sort_values(ascending=False).index[:10].to_list()

In [68]:
#for label in top_10:
#    df[label] = np.where(df["X2"]==label,1,0)
#df[["X2"]+top_10].head(20)

In [73]:
# Now we can make function to do this task for all the categorical variables
def top_n_categories(df,variable,top_n):
    top_n_categories = df[variable].value_counts().sort_values(ascending=False).index[:top_n].to_list()
    for label in top_n_categories:
        df[variable+"_"+label] = np.where(df[variable]==label,1,0)
    return df

# Checking all the categorical columns in ou dataframe
categorical_cols = [column for column in df.columns if df[column].dtype == "O"]

# Creating all the One hot encoded variables
for col in categorical_cols:
    top_n_categories(df,col,5)

df.shape

(4209, 35)

## Advantages 
1. Requires less amount of time
2. saves feature space
## Disadvantage
1. Does'nt add any new info to variable that can improve accuracy
2. And we are loosing some amount of information here 
3. **Importantly** This code is prone to error when there are not enough categories in the variable and you are asking for n categories

## Other Approaches
* **Target Encoding** ---> Where the categories are encoded as the mean of the target variable respectively fro each category
    * One problem is that it decodes categorical variable based on the target only and can cause problems with other
    * Rare categories---> in this case mean can or can not represent the category clearly
        * We can use smooting using m smoothin larger values of m will be useful in case of rare categories    
                        from category_encoders import MEstimateEncodercan be used
* **Bayesian Target Encoding** is a more mathematically involved approach towards using the target as an encoding method. Using only the mean can be a deceiving metric, so Bayesian target encoding seeks to incorporate other statistical measures of the target variable’s distribution, such as its variance or its skewness — referred to as ‘higher moments’.
 

## Probability ratio (Odds) Encodicng

* We do it in the case of y variable being the binary categorical variable

In [84]:
  df = pd.read_csv("Data/benz/train.csv",usecols=["X1","X2","X3","X4","X5","X6","y"])

In [85]:
df.head()

,y,X1,X2,X3,X4,X5,X6
0,130.81,v,at,a,d,u,j
1,88.53,t,av,e,d,y,l
2,76.26,w,n,c,d,x,j
3,80.62,t,n,f,d,x,l
4,78.02,v,n,f,d,h,d


________________________________________

# Q2 Why do we need to perform feature scaling

* Each numerical variable data have units and magnitude. The unit part determines the numeric value for the data and when we are talking about many variables we see that the units of them can not be same (for weight and height) so we can actually compare the two variables.
But some machine learning alsorithms especially eucledian distance based algo like (KNN and KMeans) tends to compare the vectors of these variables using the distance between the two vectors and there it can make problem to not have the same unit.
* So to ignore such errors in the modelling we scale our feature.
For some algorithms its necessary to scale while in others it is beneficial to scale for the effectiveness of the algorith eg. in Linear Regression if we scale our variables in such a way that it doesn have to do that many iteration or the ranomely selected weights vector is already near to the global minimum in gradient descent
* In Some algos like Decision tree, Random Forest and XGboost we don't reqire it